In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend, layers, metrics
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Lambda

In [2]:
# Define a function to create the encoder model
def get_encoder(input_shape):
    """ Returns the image encoding model """
    encoder = Sequential([
        # Create the encoder layers based on the summary you provided
        tf.keras.applications.Xception(
            input_shape=input_shape,
            weights=None,  # Don't load weights here, will load saved weights later
            include_top=False,
            pooling='avg',
        ),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    return encoder


In [3]:
# Create an instance of the encoder model
encoder_obj = get_encoder((128, 128, 3))

In [4]:
# Load the saved weights into the encoder
encoder_obj.load_weights("encoder_folder/encoder")


# Prediction based on encoder

In [5]:
# # Define a function to preprocess and encode an image
# def encode_image(image_path, encoder):
#     # Load and preprocess the image
#     image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
#     image = tf.keras.preprocessing.image.img_to_array(image)
#     image = tf.keras.applications.xception.preprocess_input(image)
#     image = tf.expand_dims(image, axis=0)  # Add batch dimension

#     # Encode the image using the encoder
#     encoded_image = encoder.predict(image)
#     return encoded_image


In [6]:
# # Define a function to compute the distance between two encoded images
# def compute_distance(encoded_image1, encoded_image2):
#     distance = tf.norm(encoded_image1 - encoded_image2, axis=1)
#     return distance


In [7]:
# # Example usage:
# image_path1 = "dog_data\\5907\\5907_A_h-8AQKUCJ6wAAAAAAAAAAAAAAQAAAQ.jpg"
# image_path2 = "dog_data\\5907\\5907_A_TKDrRJ6csBwAAAAAAAAAAAAAAQAAAQ.jpg"

# # Encode the images
# encoded_image1 = encode_image(image_path1, encoder)
# encoded_image2 = encode_image(image_path2, encoder)

# # Compute the distance between the encoded images
# distance = compute_distance(encoded_image1, encoded_image2)

# # Threshold for deciding similarity
# threshold = 0.8  # You may adjust this threshold based on your task

# # Check if the distance is below the threshold
# if distance < threshold:
#     print("The images are of same Dog.")
# else:
#     print("The images are not same of Same Dog.")

1/1 [==============================] - 0s 131ms/step
The images are similar.


# Loading Model 

In [5]:
class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


def get_siamese_network(encoder, input_shape = (128, 128, 3)):

    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")

    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)

    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
        encoded_a,
        encoded_p,
        encoded_n
    )
    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

In [6]:
siamese_network = get_siamese_network(encoder=encoder_obj)
siamese_network.summary()

Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Anchor_Input (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Positive_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Negative_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                                  

In [7]:
# Now, we'll define the SiameseModel class and create an instance of it:
class SiameseModel(Model):
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()
        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        return [self.loss_tracker]

In [8]:
siamese_model = SiameseModel(siamese_network)

In [9]:
siamese_model.load_weights("siamese_model_folder/siamese_model-final")

In [10]:
siamese_network.summary()

Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Anchor_Input (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Positive_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Negative_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                                  

In [14]:
# Function to load and preprocess an image
import cv2
def load_image(image_path, target_size=(128, 128)):
    # Load the image from the specified path
    image = cv2.imread(image_path)
    # Resize the image to the target size
    image = cv2.resize(image, target_size)
    # Convert the image from BGR to RGB format (OpenCV loads images in BGR format by default)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Normalize the image to the range [0, 1]
    image = image / 255.0
    return image


In [21]:
# Function to get predictions for new data pairs
def predict_similarity(image_path1, image_path2):
    # Load and preprocess the images
    image1 = load_image(image_path1)
    image2 = load_image(image_path2)
    pair = np.array([image1, image2])

    # Reshape the input pair to match the model's input shape
    pair = np.expand_dims(pair, axis=0)

    # Get the prediction from the model
    predictions = siamese_model.predict([pair[:, 0], pair[:, 1], pair[:, 1]])

    # Check if the distances are within a certain threshold to determine similarity
    threshold = 0.5  # Adjust this threshold as needed
    similarity = "Similar" if predictions[0] < threshold else "Different"
    return similarity

In [28]:
# Example usage:
image1 = "dog_data\\5959\\5959_A_lPwKS7HXTIUAAAAAAAAAAAAAAQAAAQ.jpg"  # Load your image here
image2 = "dog_data\\5959\\5959_A_p1FURL5LiRoAAAAAAAAAAAAAAQAAAQ.jpg" # Load your image here

result = predict_similarity(image1, image2)
print("Prediction:", result)

1/1 [==============================] - 0s 278ms/step
Prediction: Similar
